In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from keras.preprocessing import image

from collections import defaultdict

import time
import os
from PIL import Image
#Hash images using the ImageHash library
#https://pypi.python.org/pypi/ImageHash
import imagehash

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
time.time()

1517720559.493251

In [3]:
time.time()

1517720566.3615081

In [4]:
type(time.time())

float

In [5]:
lst = ['a', 'b', 'c']
lst[0:1]

['a']

### Prototype what I want ImageManager interface to look like

In [ ]:
manager = ImageManager('~/tree-logic/tree_photos/')
directory_names = ['picea_sitchensis', 'betula_papyrifera']
#manager.take_inventory(directory_names) #
#or manager.take_inventory() #default is to walk through all subdirectories
manager.sync_images(directory_names) #Syncs the manager's dataframe and dictionary to the specified directories
#or manager.sync_images() #default is to sync all subdirectories

manager.get_duplicates() #get dictionary of image hashes mapping to more than one path
manager.remove_duplicates() #delete duplicate image files

### Load a dataframe to figure out how to select certain things

In [8]:
image_df = pd.read_csv('data/tree_image_info.csv', sep='|', index_col=0)
image_df.head()

,p_hash,filename,species,tags
0,99a7465a1e99acb4,image_picea_sitchensis_27.png,picea_sitchensis,NaN
1,93d12f8e712ef068,image_picea_sitchensis_33.png,picea_sitchensis,NaN
2,cce3482eb991533b,image_picea_sitchensis_6.png,picea_sitchensis,NaN
3,c119991c4fcf1cda,image_picea_sitchensis_in_winter_12.png,picea_sitchensis,NaN
4,e3c3dab100fdc29c,image_sitka_spruce_59.png,picea_sitchensis,NaN


In [9]:
image_df['species'].unique()

array(['picea_sitchensis', 'platanus_acerifolia', 'acer_macrophylum'], dtype=object)

In [11]:
platanus_slice_df = image_df.loc[image_df['species'] == 'platanus_acerifolia']
platanus_slice_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 335 entries, 309 to 643
Data columns (total 4 columns):
p_hash      335 non-null object
filename    335 non-null object
species     335 non-null object
tags        0 non-null float64
dtypes: float64(1), object(3)
memory usage: 13.1+ KB


In [12]:
pd.Timestamp.now()

Timestamp('2018-02-04 20:41:50.969472')

In [13]:
platanus_slice_df['timestamp'] = pd.Timestamp.now()

/Users/ndbs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
platanus_slice_df.head()

,p_hash,filename,species,tags,timestamp
309,b95f8bc2719411d9,image_london_plane_36.png,platanus_acerifolia,NaN,2018-02-04 20:42:37.365247
310,e1e61e94e95a84e9,image_platanus_acerifolia_32.png,platanus_acerifolia,NaN,2018-02-04 20:42:37.365247
311,b0b1c75f4e8f4c30,image_platanus_acerifolia_6.png,platanus_acerifolia,NaN,2018-02-04 20:42:37.365247
312,cc9639a1fb5407e8,image_platanus_acerifolia_26.png,platanus_acerifolia,NaN,2018-02-04 20:42:37.365247
313,aa47f62eacd85286,image_platanus_acerifolia_in_winter_32.png,platanus_acerifolia,NaN,2018-02-04 20:42:37.365247


In [17]:
image_df.loc[309:330]

,p_hash,filename,species,tags
309,b95f8bc2719411d9,image_london_plane_36.png,platanus_acerifolia,NaN
310,e1e61e94e95a84e9,image_platanus_acerifolia_32.png,platanus_acerifolia,NaN
311,b0b1c75f4e8f4c30,image_platanus_acerifolia_6.png,platanus_acerifolia,NaN
312,cc9639a1fb5407e8,image_platanus_acerifolia_26.png,platanus_acerifolia,NaN
313,aa47f62eacd85286,image_platanus_acerifolia_in_winter_32.png,platanus_acerifolia,NaN
314,b28851b35da5ce1e,image_platanus_acerifolia_in_winter_26.png,platanus_acerifolia,NaN
315,c39736f60cc85d23,image_platanus_acerifolia_fruit_44.png,platanus_acerifolia,NaN
316,f82b82f007cdf654,image_platanus_acerifolia_fruit_50.png,platanus_acerifolia,NaN
317,e6ad99019dc219de,image_platanus_acerifolia_fruit_1.png,platanus_acerifolia,NaN
318,fe5a85b11c273166,image_platanus_acerifolia_fruit_78.png,platanus_acerifolia,NaN


In [18]:
timestamp1 = pd.Timestamp.now()
timestamp1

Timestamp('2018-02-04 20:45:53.454133')

In [19]:
timestamp2 = pd.Timestamp.now()

In [20]:
timestamp1 < timestamp2

True

In [21]:
timestamp1 > timestamp2

False

In [22]:
timestamp1 - timestamp2

Timedelta('-1 days +23:59:45.105714')

In [23]:
timestamp2 - timestamp1

Timedelta('0 days 00:00:14.894286')

In [25]:
platanus_slice_df[platanus_slice_df['timestamp']>timestamp1]

,p_hash,filename,species,tags,timestamp


In [26]:
image_dict = {h: [filename, species] for h, filename, species
              in zip(image_df['p_hash'], image_df['filename'], image_df['species'])
              if species in ['picea_sitchensis', 'platanus_acerifolia']}

In [29]:
len(image_dict)

644

In [36]:
image_df.loc[image_df['species'].isin(['platanus_acerifolia', 'acer_macrophylum']), 'p_hash'].describe()

count                  705
unique                 705
top       85efeac7c6413344
freq                     1
Name: p_hash, dtype: object

In [42]:
both = set(image_dict.keys()).intersection(image_df.loc[image_df['species'].isin(['platanus_acerifolia', 'acer_macrophylum']), 'p_hash'])

In [43]:
len(both)

335

In [45]:
image_df.loc[image_df['p_hash'].isin(both), 'species'].unique() #Woo hoo, this is the expected answer

array(['platanus_acerifolia'], dtype=object)